In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('roskomvsoboda_chlb_merge.csv')

In [10]:
def tweak_decnum_scrapping(df):
    df_ = (df
           .loc[:, ['record_id', 'decisionNumber', 'decisionDate', 
                    'authority_id', 'authority_name', 'status',
                    'domains', 'urls']]
          )
    return df_

df = tweak_decnum_scrapping(df)#.to_csv('roskomvsoboda_chlb_scrapping.csv', index=False)

In [12]:
df_2021 = pd.read_csv('roskomvsoboda_chlb_scrapping_2021.csv')

df_con = pd.concat([df_2021, df], ignore_index=True)
df_con

,record_id,decisionNumber,decisionDate,authority_id,authority_name,status,domains,urls
0,2904472,б/н,2021-01-15,103,Центральный районный суд г. Челябинска - Челяб...,blocked,['prostitutkikurganagirls.com'],[]
1,2924065,б/н,2021-01-15,103,Центральный районный суд г. Челябинска - Челяб...,blocked,['prostitutkikurganabest.com'],[]
2,2924066,б/н,2021-01-15,103,Центральный районный суд г. Челябинска - Челяб...,blocked,['sex-portal.org'],[]
3,2961133,2а-14/20,2021-01-19,714,Нязепетровский районный суд - Челябинская область,blocked,['xx-74.ru'],[]
4,2966748,2а-12/2021,2021-01-19,714,Нязепетровский районный суд - Челябинская область,blocked,['chelyabinsk.diplomnarukah.com'],[]
...,...,...,...,...,...,...,...,...
2125,5496088,2а-1212/2022,2022-12-29,1836,Карталинский городской суд - Челябинская область,blocked,['socuteporn.com'],[]
2126,5496540,2а-1210/2022,2022-12-29,1836,Карталинский городской суд - Челябинская область,blocked,['www.ruseks.top'],[]
2127,5524205,2а-8096/2022,2022-12-14,66,Курчатовский районный суд г. Челябинска - Челя...,blocked,['med.spravki-za-chas.top'],['https://med.spravki-za-chas.top/product/medi...
2128,5529744,2а-8097/2022,2022-12-14,66,Курчатовский районный суд г. Челябинска - Челя...,blocked,['spravkak-magazine.com'],['https://spravkak-magazine.com/spravka-na-oru...


In [8]:
df_con.decisionNumber.value_counts()

б/н                           35
2а-3820/2022                  13
2a-794/2022                   11
2а-891/2022                    7
2а-163/2021                    7
                              ..
2a-2689/2022                   1
2a-2688/2022                   1
2a-785/2022 (2а-5029/2021)     1
2a-2612/2022                   1
2а-8097/2022                   1
Name: decisionNumber, Length: 1592, dtype: int64

In [18]:
df_con.loc[df_con.decisionNumber == '2a-2689/2022']

,record_id,decisionNumber,decisionDate,authority_id,authority_name,status,domains,urls
698,4119250,2a-2689/2022,2022-02-17,103,Центральный районный суд г. Челябинска - Челяб...,blocked,['cosplay-porn.me'],[]


Extract domain name. 

In [34]:
def generalize_topn(ser, n=10, other='Other'):
    topn = ser.value_counts().index[:n]
    if isinstance(ser.dtype, pd.CategoricalDtype):
        ser = ser.cat.set_categories( 
            topn.set_categories(list(topn)+[other]))
    return ser.where(ser.isin(topn), other)

In [75]:
df_dom_analysis = (df_con
                 .assign(domain_register=lambda df_: df_.domains.str.extract(r"^\['\S+(\.\w{1,6})'\]$", expand=False),
                         decision_month=lambda df_: df_.decisionDate.str[:7],
                         decision_year=lambda df_: df_.decisionDate.str[:4])
                 .loc[:, ['record_id', 'decisionNumber', 'decisionDate', 'decision_month', 
                          'decision_year', 'domains', 'domain_register', 'authority_name']]
                )

df_dom_analysis

,record_id,decisionNumber,decisionDate,decision_month,decision_year,domains,domain_register,authority_name
0,2904472,б/н,2021-01-15,2021-01,2021,['prostitutkikurganagirls.com'],.com,Центральный районный суд г. Челябинска - Челяб...
1,2924065,б/н,2021-01-15,2021-01,2021,['prostitutkikurganabest.com'],.com,Центральный районный суд г. Челябинска - Челяб...
2,2924066,б/н,2021-01-15,2021-01,2021,['sex-portal.org'],.org,Центральный районный суд г. Челябинска - Челяб...
3,2961133,2а-14/20,2021-01-19,2021-01,2021,['xx-74.ru'],.ru,Нязепетровский районный суд - Челябинская область
4,2966748,2а-12/2021,2021-01-19,2021-01,2021,['chelyabinsk.diplomnarukah.com'],.com,Нязепетровский районный суд - Челябинская область
...,...,...,...,...,...,...,...,...
2125,5496088,2а-1212/2022,2022-12-29,2022-12,2022,['socuteporn.com'],.com,Карталинский городской суд - Челябинская область
2126,5496540,2а-1210/2022,2022-12-29,2022-12,2022,['www.ruseks.top'],.top,Карталинский городской суд - Челябинская область
2127,5524205,2а-8096/2022,2022-12-14,2022-12,2022,['med.spravki-za-chas.top'],.top,Курчатовский районный суд г. Челябинска - Челя...
2128,5529744,2а-8097/2022,2022-12-14,2022-12,2022,['spravkak-magazine.com'],.com,Курчатовский районный суд г. Челябинска - Челя...


Make simple substring-based filter to remove the most obvious non-fitting cases.

In [98]:
stop_words = ['audio', 'video', 'sliv', 'kino', 'film', 'tv', 'movie', 'resheba', 'books', 'sex',
              'klad', 'nark', 'torrent', 'muz', 'knigi', 'hd', 'serial', 'series', 'tracker',
              'bill', 'prostitutk', 'porn', 'intim', 'seks', 'amur', 'feiportal', 'kurtizanki', 'dosug',
              'shluh', 'xx', 'murmur', 'diplom', 'prava', 'dublikat', 'casino', 'devki', 'soski', 'parnuha',
              'boobs', 'eccemoda', 'oruzhie', 'love', 'tube', 'lady', 'spravk', 'putan', 'damochki', 'doska',
              'devochki', 'obnal', 'dokument', 'pvkgb', 'guns', 'document', 'slut', 'prostutytku', 'kiss',
              'whore', 'otsos', '4zzy', 'transi', 'sprawka', 'individualka', 'titki', 'pizd', 'siski', 'ebu',
              'doc', 'golye', 'escort', 'dildos', 'harlots']

(df_dom_analysis
 .loc[~df_dom_analysis.domains.str.contains("|".join(stop_words), regex=True)]
 .sort_values('decisionDate')
)#.to_csv('chelyabinsk_2021_2022_cases.csv', index=False)

,record_id,decisionNumber,decisionDate,decision_month,decision_year,domains,domain_register,authority_name
21,3049955,2а-418-2021,2021-02-17,2021-02,2021,['thomas-stege.de'],.de,Саткинский городской суд - Челябинская область
616,4343144,2а-1289/2021,2021-02-19,2021-02,2021,['admiralwin.fun'],.fun,Курчатовский районный суд г. Челябинска - Челя...
117,3289057,2а-629/2021,2021-02-20,2021-02,2021,['piholpua.ds216.ru'],.ru,Правобережный районный суд г. Магнитогорска - ...
116,3288196,2а-631/2021,2021-02-20,2021-02,2021,['kesalon.victoria-rossi.ru'],.ru,Правобережный районный суд г. Магнитогорска - ...
60,3139653,2а-768/2021,2021-02-24,2021-02,2021,['playfortunagl12.com'],.com,Металлургический районный суд г. Челябинска - ...
...,...,...,...,...,...,...,...,...
1904,5242102,2а-815/2022 ~ М-812/2022,2022-12-23,2022-12,2022,['snlines.shop'],.shop,Нагайбакский районный суд - Челябинская область
2008,5278489,2а-729/2022 ~ М-698/2022,2022-12-26,2022-12,2022,['dokhumentn-ohrana.com'],.com,Усть-Катавский городской суд - Челябинская обл...
2103,5455005,2а-2622/2022,2022-12-28,2022-12,2022,['ozersk.soos.su'],.su,Кыштымский городской суд - Челябинская область
2111,5459899,2а-1217/2022,2022-12-29,2022-12,2022,['chelyabinsk.extern-dip.com'],.com,Карталинский городской суд - Челябинская область
